In [7]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector
from mysql.connector import Error
from urllib.parse import urlparse

# -----------------------------
# Step 1: Determine which month to fetch
# -----------------------------
def get_report_month():
    today = datetime.now()
    # If today < 15th, use 2 months back; else previous month
    if today.day < 15:
        report_month = today.replace(day=1) - timedelta(days=32) 
    else:
        report_month = today.replace(day=1) - timedelta(days=1)
    return report_month.strftime("%b").lower(), report_month.strftime("%Y")

# -----------------------------
# Step 2: Generate report URL
# -----------------------------
def get_base_url():
    month_name, year = get_report_month()
    return f'https://portal.amfiindia.com/spages/am{month_name}{year}repo.xls', month_name.capitalize(), year

# -----------------------------
# Step 3: Download the report
# -----------------------------
def download_report(folder_path):
    try:
        base_url, month_name, year = get_base_url()
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        filename = os.path.basename(urlparse(base_url).path)
        file_path = os.path.join(folder_path, filename)

        print(f"Downloading {month_name} {year} report to: {file_path}")
        req = requests.get(base_url, stream=True)
        if req.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in req.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            print("Download complete.")
            return file_path, month_name, year
        else:
            print(f"Failed to download: HTTP status code {req.status_code}")
            return None, month_name, year
    except Exception as e:
        print(f"Error during download: {e}")
        return None, None, None

# -----------------------------
# Step 4: Process and clean report
# -----------------------------
def process_report(file_path):
    try:
        df = pd.read_excel(file_path, skiprows=1)

        df.columns = [str(col).strip() for col in df.columns]

        new_header_names = ['SR','Scheme_Name','No_of_Scheme','No_of_Folio', 
                            'Gross_Sales','Redemption','Net_Sales','AUM','AAUM','No_of_Portfolio','NAV']
        df.columns = new_header_names

        df['SR'] = df['SR'].astype(str)

        scheme_type_map = {'A': 'Open Ended Scheme', 'B': 'Closed Ended Scheme', 'C': 'Interval Scheme'}
        detailed_scheme_map = {
            'I': 'Income/Debt Oriented Schemes',
            'II': 'Growth/Equity Oriented Schemes',
            'III': 'Hybrid Schemes',
            'IV': 'Solution Oriented Schemes',
            'V': 'Other Schemes'
        }

        current_scheme = None
        current_detailed_scheme = None

        def get_scheme_type(x):
            nonlocal current_scheme
            if x in scheme_type_map:
                current_scheme = scheme_type_map[x]
            return current_scheme

        def get_detailed_scheme(x):
            nonlocal current_detailed_scheme
            if x in detailed_scheme_map:
                current_detailed_scheme = detailed_scheme_map[x]
            return current_detailed_scheme

        df['Scheme_Type'] = df['SR'].apply(get_scheme_type)
        df['Detailed_Scheme_Type'] = df['SR'].apply(get_detailed_scheme)

        df['Scheme_Type'] = df['Scheme_Type'].ffill()
        df['Detailed_Scheme_Type'] = df['Detailed_Scheme_Type'].ffill()

        df = df.dropna(subset=["Scheme_Name"])

        unwanted_strings = ['Sub Total', 'Total', 'Grand Total',]
        df = df[~df['Scheme_Name'].astype(str).apply(lambda x: any(sub in x for sub in unwanted_strings))]

        int_cols = ['No_of_Scheme','No_of_Folio','No_of_Portfolio']
        for col in int_cols:
            df[col] = df[col].astype(str).str.replace(r'[^0-9]', '', regex=True)
            df[col] = df[col].replace('', '0').astype(int)

        float_cols = ['Gross_Sales','Redemption','Net_Sales','AUM','AAUM','NAV']
        for col in float_cols:
            df[col] = df[col].astype(str).str.replace(r'[^0-9.\-]', '', regex=True)
            df[col] = df[col].replace('', '0').astype(float)

        df = df.where(pd.notnull(df), None)

        return df
    except Exception as e:
        print(f"Error processing report: {e}")
        return None

# -----------------------------
# Step 5: Test MySQL connection
# -----------------------------
def test_mysql_connection(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=3306
        )
        if connection.is_connected():
            connection.close()
            return True
    except Error as e:
        print(f"MySQL connection error: {e}")
        return False

# -----------------------------
# Step 6: Upload DataFrame to MySQL
# -----------------------------
def upload_to_mysql(host, user, password, database, df):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=3306
        )
        cursor = connection.cursor()
        sql = """
        INSERT INTO amfi_reports (
            SR, Scheme_Name, No_of_Scheme, No_of_Folio, Gross_Sales, Redemption, 
            Net_Sales, AUM, AAUM, No_of_Portfolio, NAV, Scheme_Type, Detailed_Scheme_Type
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        # Replace NaN with None
        df = df.where(pd.notnull(df), None)

        values = [
            (
                row['SR'], row['Scheme_Name'], row['No_of_Scheme'], row['No_of_Folio'], 
                row['Gross_Sales'], row['Redemption'], row['Net_Sales'], row['AUM'], 
                row['AAUM'], row['No_of_Portfolio'], row['NAV'], row['Scheme_Type'], row['Detailed_Scheme_Type']
            ) for _, row in df.iterrows()
        ]
        cursor.executemany(sql, values)
        connection.commit()
        print(f"Inserted {cursor.rowcount} rows into 'amfi_reports'.")
        df.to_excel(r'C:\project\amfi_reports_updated.xlsx', index=False)
    except Error as e:
        print(f"Error uploading data: {e}")
    finally:
        if cursor:
            cursor.close()
        if connection and connection.is_connected():
            connection.close()

# -----------------------------
# Step 7: Main script
# -----------------------------
folder_path = r'C:\project'
host = 'localhost'
user = 'root'
password = 'mksomkso'
database = 'Gdb0041'

if test_mysql_connection(host, user, password, database):
    downloaded_file, month_name, year = download_report(folder_path)
    if downloaded_file:
        df_transformed = process_report(downloaded_file)
        if df_transformed is not None:
            print(f"Uploading {month_name} {year} report...")
            upload_to_mysql(host, user, password, database, df_transformed)

else:
    print("Failed to connect to MySQL. Please check your connection details and ensure MySQL is running.")


Download complete.
Uploading Oct 2025 report...
Inserted 63 rows into 'amfi_reports'.
